In [1]:
#Imports 

In [1]:
from pymongo import MongoClient
from dotenv import load_dotenv
import os
import pandas as pd
import openai
import nest_asyncio
nest_asyncio.apply()
import tensorflow_hub as hub
import datetime
import tensorflow as tf

from marktrix.helpers import Helper
from marktrix.calendar import Calendar, global_calendar
from marktrix.user_data import UserData
from marktrix.website_info import WebsiteInfo

pd.set_option('display.max_rows', None)

# Load your API key from an environment variable or secret management service
openai.api_key = "sk-9GD2atPAt9M1Od4YaPWIew7J3BNSlWKiO1d0Z7z3"

#load universal sentence encoder
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embed = hub.load(module_url)

In [2]:
##Globals

In [3]:
load_dotenv()

True

In [4]:
mongo_url = os.environ.get("MARKTRIX_MONGO")
client = MongoClient(mongo_url)
db = client['marktrix-production']


In [6]:
#a function that return max events embed similarity to user and target audience 
def predict_best_calendar_events(user_info, global_calendar, start_date, end_date, n_best):
    #user and website info
    company_name = Helper.get_object_field(user_info, 'CompanyName')
    company_industry = Helper.get_object_field(user_info, 'OwnIndustry')
    site_desc = Helper.get_object_field(user_info, 'SiteDescription')
    sol_desc = Helper.get_object_field(user_info, 'SolutionDescription')
    num_one_benefit = Helper.get_object_field(user_info, 'NumberOneBenefit')
    company_size = Helper.get_object_field(user_info, 'OwnCompanySize') 
    #product_categories = Helper.list_to_text(Helper.get_object_field(user_info, 'MainProductCategories'))
    #business_model = Helper.get_object_field(user_info, 'BusinessModel')  # for Tone
    #business_model = Helper.business_model_desc(business_model)
    #how_customer_deal_with_needs = Helper.get_object_field(user_info, 'HowCustomerDealWithNeeds')
    #sol_change = Helper.get_object_field(user_info, 'WhatYourSolutionChange')
    #why_sol_important = Helper.get_object_field(user_info, 'WhySolutionImportant')
    company_text = company_name + ' ' + company_industry + ' ' + site_desc + ' ' + sol_desc + ' ' + num_one_benefit +  ' ' + company_size 
                 # + '\nBusiness Model\n' + business_model \
                  # + '\nProduct Categories\n' + product_categories  \
   
    
    # target customer info
    target_customer_description = Helper.get_object_field(user_info, 'TargetCustomerDescription')
    positive_trends = Helper.get_object_field(user_info, 'PositiveTrends')
    
    target_customer_ages = Helper.get_object_field(user_info, 'TargetCustomerAges')
    if target_customer_ages != '' and type(target_customer_ages) == list:
        target_customer_ages = [Helper.customer_age_to_text(age) for age in target_customer_ages]
        target_customer_ages = Helper.list_to_text(target_customer_ages)

    target_customer_gender = Helper.get_object_field(user_info, 'TargetCustomerGender')
    if target_customer_gender != '' and type(target_customer_gender) == list:
        target_customer_gender = Helper.list_to_text(target_customer_gender)

    target_customer_geo = Helper.get_object_field(user_info, 'TargetCustomerGeography')
    if target_customer_geo != '' and type(target_customer_geo) == list:
        target_customer_geo = Helper.list_to_text(target_customer_geo)

     # decide if to use old or new
    if target_customer_ages or target_customer_gender or target_customer_geo:
        target_customer_description = target_customer_ages + ' '  + target_customer_gender+ ' ' + target_customer_geo
    
    
    target_customer_and_trends_text = target_customer_description + ' ' + positive_trends
   
    #create embeddings from company's text and target customers
    company_embed = embed(company_text)
    
    #If target customer text and trends is empty use Company_text
    if  target_customer_and_trends_text is not '':
        target_customers_embed = embed(target_customer_and_trends_text)
    else: 
        target_customers_embed = embed(company_text)

    
    # will hold info for each date
    calendar_d = global_calendar.cdays
    calendar_m = global_calendar.mdays
    calendar_m = pd.DataFrame(calendar_m)
    calendar_d = pd.DataFrame(calendar_d)
    all_events = pd.concat([calendar_m, calendar_d])
    all_events = all_events[['name','desc', 'date']]
    all_events['date'] = pd.to_datetime(all_events['date'])
    all_events['date'] = all_events['date'].dt.date
    all_events = all_events.drop_duplicates(subset=['name'])
    mask = (all_events['date'] > start_date) & (all_events['date'] <= end_date)
    concurrent_events = all_events.loc[mask]
    concurrent_events_name = concurrent_events['name'] 
    concurrent_events_desc = concurrent_events['desc'] 
    concurrent_events['text'] = concurrent_events_name + ' ' + concurrent_events_desc
    #create embeddings from concurrent events texts 
    concurrent_events_embed = embed(concurrent_events['text'])
   
    #calculate target customer embed correlation with concurrent_events_embed 
    target_event_cos_sim =  tf.keras.losses.cosine_similarity(concurrent_events_embed, target_customers_embed)
    #target_cosine_sim = pd.DataFrame(data = target_event_cor)
    

    #calculate company embed correlation with concurrent_events_embed
    company_event_cos_sim =  tf.keras.losses.cosine_similarity(concurrent_events_embed, company_embed)
    #company_cosine_sim = pd.DataFrame(data = company_event_cor)

    #calculates a similarity score based on combination of company and target customers similarity scores
    similarity_score = target_event_cos_sim*0.8 + company_event_cos_sim*0.2
    concurrent_events['similarity_score'] =  similarity_score
    df = concurrent_events.nlargest(n_best, 'similarity_score')
    df = df.sort_values(by=['similarity_score'], ascending=False)
    
    df= df[['name','desc', 'date','similarity_score']]
    
    return df 
    #with dates (can be index), pred, event. - sorted top pred first, event is event object as in calendar (day or month)

<>:49: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:49: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\One1\AppData\Local\Temp\ipykernel_19504\2458016291.py:49: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if  target_customer_and_trends_text is not '':
